# Public Health in Singapoore

## Data Analysis

In [1]:
from visuals import *

### Life Expectancy

In [2]:
plot_life_expectancy()

### Ageing Population

Geriatric Medicine is provided for those aged 75 and above.

In [3]:
plot_ageing_population()

### Childhood Obesity

In [4]:
plot_obesity_rate()

### Common Health Conditions

In [5]:
plot_conditions()

### Top Causes of Death

In [6]:
plot_deaths()

## Data Sources

1. SingStat Table Builder
    - [Singapore Residents By Age Group, Ethnic Group And Sex, End June](https://tablebuilder.singstat.gov.sg/table/TS/M810011)
    - [Overweight And Severely Overweight (Obesity) Prevalence Among Primary One Children](https://tablebuilder.singstat.gov.sg/table/TS/M870381)
    
2. Data.gov.sg
    - [Top 4 Conditions of Polyclinic Attendances](https://data.gov.sg/dataset/top-4-conditions-of-polyclinic-attendances)
    - [Number of Deaths and Top 10 Principal Causes](https://data.gov.sg/dataset/principal-causes-of-death)
    - [Health Adjusted Life Expectancy at Birth](https://data.gov.sg/dataset/health-adjusted-life-expectancy)